# Common SNPs per gene in ExAC

In [ ]:
# url = 'ftp://ftp.broadinstitute.org/pub/ExAC_release/release0.3/ExAC.r0.3.sites.vep.vcf.gz'
# ! wget --timestamping --no-verbose --directory-prefix download {url}

In [ ]:
import re
import numpy
import pandas
import vcf

## Helper functions

In [ ]:
pattern = re.compile('ENSG[0-9]+')

def genes_in_csq(csq):
    ensembl_genes = set()
    for x in csq:
        ids = re.findall(pattern, x)
        ensembl_genes.update(ids)
    return ensembl_genes

In [ ]:
def get_allele_frequency(x):
    """x is list or array"""
    x = numpy.array(x)
    return x[(numpy.abs(x - 0.5)).argmin()]

## Parse

In [ ]:
# Options
path = 'download/ExAC.r0.3.sites.vep.vcf.gz'
maj_af_min = 0.05

In [ ]:
rows = []
i = 0
for r in vcf.Reader(filename=path):
    
    # Quality control
    if r.FILTER:
        continue
    
    # Exclude non-SNPs
    if not r.is_snp:
        continue
    
    # Major allele frequency check
    allele_freq = get_allele_frequency(r.INFO['AF'])
    if not ((allele_freq >= maj_af_min) and
            (allele_freq <= 1 - maj_af_min)):
        continue

    # Extract genes
    genes = genes_in_csq(r.INFO.get('CSQ', []))

    # Add to rows
    row = r.CHROM, r.POS, allele_freq
    for gene in genes:
        rows.append(row + (gene,))
    i += 1
    if i > 100:
        break

exac_df = pandas.DataFrame(rows, columns=['chromosome', 'position', 'allele_freq', 'ensembl_gene_id'])
exac_df.head(3)

In [ ]:
len(exac_df)

In [ ]:
exac_df.to_csv('data/exac-filtered.tsv', index=False, sep='\t')

In [ ]:
# Read Ensembl to Entrez gene mapping
url = 'https://raw.githubusercontent.com/dhimmel/entrez-gene/6e133f9ef8ce51a4c5387e58a6cc97564a66cec8/data/xrefs-human.tsv'
entrez_map_df = pandas.read_table(url)
entrez_map_df = entrez_map_df.query("resource == 'Ensembl'")
entrez_map_df = entrez_map_df[['GeneID', 'identifier']]
entrez_map_df = entrez_map_df.rename(columns={'GeneID': 'entrez_gene_id', 'identifier': 'ensembl_gene_id'})
entrez_map_df.head(2)

In [ ]:
exac_df = exac_df.merge(entrez_map_df)

In [ ]:
def get_snps_per_gene(df):
    df = df.drop_duplicates(['chromosome', 'position'])
    return pandas.Series({'snps': len(df)})

count_df = exac_df.groupby('entrez_gene_id').apply(get_snps_per_gene).reset_index()
count_df.head(2)

In [ ]:
count_df.to_csv('data/exac-counts.tsv', index=False, sep='\t')